In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

pd.set_option('future.no_silent_downcasting', True)






In [229]:
def gradient(weights, featureMatrix, labels, dataNum):
    sum=np.zeros(weights.shape,dtype=float)
    
    for i in range(1):
        num = np.dot(-1 * labels[i], featureMatrix[i])

        label = np.float64(labels[i])
        print(weights, featureMatrix[i])
        dotP = np.array(np.dot(weights, featureMatrix[i]))[0]
        print(dotP)
        denom = 1 + np.exp(label * dotP)#(1,14) * (14,1)
        print(denom)
        sum+= np.array((1/denom) * np.array(num))
    return sum/dataNum



def logisticRegression(weights, featureMatrix, labels, learningRate, epochs,dataNum):
    for i in range(epochs):
        weights = weights - learningRate * gradient(weights, featureMatrix, labels,dataNum)
    return weights

def predict(xTest,yTest,newWeights):
    featureMatrix = np.hstack((xTest, np.ones((xTest.shape[0],1))))
    correct = 0
    outArray=[]
    for i in range(featureMatrix.shape[0]):
        testY = (newWeights @ featureMatrix[i])
        outArray.append(1 if testY>=0 else -1)
        if (testY) > 0:
            if yTest[i] == 1:
                correct+=1
        else:
            if yTest[i] == -1:
                correct+=1
    
    
    
    return correct/featureMatrix.shape[0]





In [206]:
df = pd.read_csv("Heart_Disease_Prediction.csv")
backUpData = df.copy()
#print(backUpData.loc[1])
df['Heart Disease'] = df['Heart Disease'].replace(['Presence','Absence'],[1,-1])

ratio = 0.9

df = df.sample(frac=1)
Train,Test = df.iloc[:int(ratio*len(df))],df.iloc[int(ratio*len(df)):]
xData = np.array(Train.iloc[:,:-1])
yData = np.array(Train.iloc[:,-1])
xTest = np.array(Test.iloc[:,:-1])
yTest = np.array(Test.iloc[:,-1])
featureMatrix = np.hstack((xData, np.ones((xData.shape[0],1))))


learningRate = 10**-6
dataNum = featureMatrix.shape[0]

weights = np.random.random((1,featureMatrix.shape[1]))#(1,14)
correctedWeights = logisticRegression(weights, featureMatrix,yData,learningRate,1000,dataNum)
print("Accuracy: ",predict(xTest,yTest,correctedWeights))





[[0.33206138 0.05624771 0.29480981 0.71128841 0.83634514 0.34691543
  0.06700503 0.64928507 0.10879745 0.71798671 0.46805114 0.1962653
  0.63964732 0.55389347]] [ 67.   1.   4. 120. 237.   0.   0.  71.   0.   1.   2.   0.   3.   1.]
[357.27817152]
1.4586073497716699e+155
[[0.33206138 0.05624771 0.29480981 0.71128841 0.83634514 0.34691543
  0.06700503 0.64928507 0.10879745 0.71798671 0.46805114 0.1962653
  0.63964732 0.55389347]] [ 67.   1.   4. 120. 237.   0.   0.  71.   0.   1.   2.   0.   3.   1.]
[357.27817152]
1.4586073497716699e+155
[[0.33206138 0.05624771 0.29480981 0.71128841 0.83634514 0.34691543
  0.06700503 0.64928507 0.10879745 0.71798671 0.46805114 0.1962653
  0.63964732 0.55389347]] [ 67.   1.   4. 120. 237.   0.   0.  71.   0.   1.   2.   0.   3.   1.]
[357.27817152]
1.4586073497716699e+155
[[0.33206138 0.05624771 0.29480981 0.71128841 0.83634514 0.34691543
  0.06700503 0.64928507 0.10879745 0.71798671 0.46805114 0.1962653
  0.63964732 0.55389347]] [ 67.   1.   4. 120. 23

In [211]:
bigDF = pd.read_excel("LargerData.xlsx")
print(bigDF.head())


   PatientID    State     Sex GeneralHealth   AgeCategory  HeightInMeters  \
0          1  Alabama  Female          Fair  Age 75 to 79            1.63   
1          2  Alabama  Female     Very good  Age 65 to 69            1.60   
2          3  Alabama    Male     Excellent  Age 60 to 64            1.78   
3          4  Alabama    Male     Very good  Age 70 to 74            1.78   
4          5  Alabama  Female          Good  Age 50 to 54            1.68   

   WeightInKilograms        BMI  HadHeartAttack  HadAngina  ...  \
0          84.820000  32.099998               0          1  ...   
1          71.669998  27.990000               0          0  ...   
2          71.209999  22.530001               0          0  ...   
3          95.250000  30.129999               0          0  ...   
4          78.019997  27.760000               0          0  ...   

                             ECigaretteUsage  ChestScan  \
0  Never used e-cigarettes in my entire life          1   
1  Never used e-

# Beginning feature elimination
### Need to clean up the following columns: AgeCategory, Sex, State, SmokerStatus

In [225]:
dataSet = bigDF[['State','Sex','AgeCategory','HeightInMeters','WeightInKilograms',
                 'BMI','HadAngina','HadStroke','HadAsthma','HadSkinCancer','HadCOPD',
                 'HadDiabetes','DifficultyWalking','SmokerStatus','AlcoholDrinkers',
                 'HighRiskLastYear','HadHeartAttack']]


states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware',
                                             'Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
                                             'Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska',
                                             'Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma',
                                             'Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia',
                                             'Washington','West Virginia','Wisconsin','Wyoming', "Puerto Rico", "District of Columbia", 'Guam', 'Virgin Islands']


dataSet['State'] = dataSet['State'].replace(states,list(range(len(states))))

ageCats = [
    'Age 65 to 69',     
'Age 60 to 64',
'Age 70 to 74',
'Age 55 to 59',
'Age 50 to 54',
'Age 75 to 79',
'Age 80 or older',
'Age 40 to 44',
'Age 45 to 49',
'Age 35 to 39',
'Age 30 to 34',
'Age 18 to 24',
'Age 25 to 29',
]
dataSet['AgeCategory'] = dataSet['AgeCategory'].replace(sorted(ageCats),list(range(len(ageCats))))

diab = [
    'No',
'Yes',
'No, pre-diabetes or borderline diabetes',
'Yes, but only during pregnancy (female)',
]

dataSet['HadDiabetes'] = dataSet['HadDiabetes'].replace(diab,[0,1,2,3])

smoke=[
    'Never smoked',
'Former smoker',
'Current smoker - now smokes every day',
'Current smoker - now smokes some days'
]

dataSet['SmokerStatus'] = dataSet['SmokerStatus'].replace(smoke,[0,1,2,3])
dataSet['Sex'] = dataSet['Sex'].replace(['Male','Female'],[0,1])

_ = [print(f"{dataSet[column].value_counts()}\n\n") for column in dataSet.columns]


State
46    14241
19     8817
22     8712
34     8700
31     8625
42     7267
8      7124
15     6000
48     5890
18     5709
14     5492
13     5393
39     5360
45     5358
2      5302
10     5262
43     5212
21     5206
20     5164
26     5008
5      4973
9      4860
4      4801
6      4765
44     4569
40     4280
25     4155
24     4042
29     3833
28     3564
50     3550
11     3394
1      3100
38     3006
17     2958
36     2907
35     2899
47     2898
30     2894
3      2881
41     2659
37     2623
32     2531
12     2516
33     2435
23     2416
16     2381
49     2345
7      2079
0      1880
27     1693
51     1648
52     1521
53      732
Name: count, dtype: int64


Sex
1    123293
0    114337
Name: count, dtype: int64


AgeCategory
9     27547
8     25685
10    24946
7     21422
6     19154
11    17679
12    17544
4     16228
5     16095
3     14982
2     12825
0     12777
1     10746
Name: count, dtype: int64


HeightInMeters
1.68    20909
1.63    20104
1.70    19356
1.78    1

/var/folders/_h/6qb3_34d6zzg_p_8t37fxdbc0000gn/T/ipykernel_46327/2340100575.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSet['State'] = dataSet['State'].replace(states,list(range(len(states))))
/var/folders/_h/6qb3_34d6zzg_p_8t37fxdbc0000gn/T/ipykernel_46327/2340100575.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSet['AgeCategory'] = dataSet['AgeCategory'].replace(sorted(ageCats),list(range(len(ageCats))))
/var/folders/_h/6qb3_34d6zzg_p_8t37fxdbc0000gn/T/ipykernel_46327/2340100575.py

In [230]:
XTrain,Xtest,YTrain,YTest = train_test_split(dataSet.iloc[:,:-1],dataSet.iloc[:,-1],test_size=0.1)

bigFeature = np.hstack((XTrain, np.ones((XTrain.shape[0],1))))


YTrain = np.array(YTrain)
dataNum = bigFeature.shape[0]

weights = np.random.random((1,bigFeature.shape[1]))#(1,14)
correctedWeights = logisticRegression(weights, bigFeature,YTrain,10**-6,1,dataNum)
print("Accuracy: ",predict(Xtest, YTest,correctedWeights))

[[0.25267199 0.50918909 0.68893027 0.69607042 0.37344548 0.64669139
  0.24355427 0.83326188 0.73680968 0.50654916 0.06349903 0.34017368
  0.48870156 0.8324314  0.67341695 0.05841723 0.30110774]] [2 1 9 1.64999997615814 77.1100006103516 28.2900009155273 0 0 0 0 0 0 0 0
 0 0 1.0]
55.755810796492234
2.0


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'